In [1]:
# Dependencies and Setup
import requests
import json
import pandas as pd

# Import the API key
from config import api_key

In [2]:
# Get the list of movie IDs from 2000 to 2024 
url = "https://api.themoviedb.org/3/discover/movie"
movie_ids = []
for year in range(2000, 2024):
    for page in range(1, 11):
        params = {
            "api_key": api_key,
            "language": "en-US",
            "sort_by": "vote_average.desc",
            "page": page,
            "primary_release_year": year,
            "vote_count.gte": 500,
        }
        response = requests.get(url, params=params)
        data = response.json()
        for movie in data["results"]:
            movie_ids.append(movie["id"])

# Put the movie IDs into a DataFrame
movies_df = pd.DataFrame(movie_ids)
print(movies_df.head())


       0
0  40096
1     98
2     77
3    843
4    641


In [3]:
# Count the number of movie IDs
print(len(movies_df))

4067


In [19]:
# Using the movie IDs, retrieve the director of each movie along with the gender and popularity
cert_list=[]

for i in movie_ids:
    movie_id = i
    url2 = f"https://api.themoviedb.org/3/movie/{movie_id}/release_dates"
    params = {
            "api_key": api_key,
        }
    response2 = requests.get(url2, params=params)
    data2 = response2.json()
    
    # The results key is an array, so we need to loop through it
    for release in data2['results']:
        if release["iso_3166_1"] == "US":
            # The release_dates key is also an array, so we need to loop through it
            for date in release['release_dates']:
                # We can check the certification and append to the list if it's not empty
                if date["type"] == 3 and date["certification"] != "":
                    # We can create the dictionary with the correct keys
                    dictionary = {"id": movie_id, "iso_3166_1": release["iso_3166_1"], "certification": date["certification"]}
                    cert_list.append(dictionary)
    
    
    # level1 = data2['results']
    # for movie in level1:
    #     if movie ["iso_3166_1"] == "US":
    #         level2 = level1['release_dates']
    #         for n in level2:
    #             if n["type"] == 3:
    #                 dictionary = {"id": data2["id"], "location": level1["iso_3166_1"]}
    #                 cert_list.append(dictionary)
#     for n in results2:
#         if n['iso_3166_1'] == 'US':
#             cert_dict = {"id": data2['id']
#                             # "certification": n['certification'],
#                             # "type": n['type'],
#                             # "release_date": n['release_date']
#                             }
#     cert_list.append(cert_dict)

# # Put the director info into a DataFrame and CSV file for loading into the SQL database
cert_df = pd.DataFrame(cert_list)
# cert_df.to_csv("certification.csv", index=False)
print(cert_df.head())

    id iso_3166_1 certification
0   98         US             R
1   98         US             R
2   77         US             R
3  843         US            PG
4  641         US             R


In [20]:
# Check the data types of the retrieved data
cert_df.to_csv("certification.csv", index=False)
print(cert_df.dtypes)

id                int64
iso_3166_1       object
certification    object
dtype: object


In [21]:
# Count the number of records in the director DataFrame
print(len(cert_df))

4066


In [22]:
# Get certification
# type 3 is Theater and type 4 is Digital, type 1 is Premiere and type 5 is Physical
cert_list=[]

for i in movie_ids:
    movie_id = i
    url2 = f"https://api.themoviedb.org/3/movie/{movie_id}/release_dates"
    params = {
            "api_key": api_key,
        }
    response2 = requests.get(url2, params=params)
    data2 = response2.json()
    
    # The results key is an array, so we need to loop through it
    for release in data2['results']:
        if release["iso_3166_1"] == "US":
            # The release_dates key is also an array, so we need to loop through it
            for date in release['release_dates']:
                # We only include release dates with iso_3166_1 equal to "US"
                if release["iso_3166_1"] == "US":
                    # If the certification is null for Type 3, include the certification for Type 4
                    if date["type"] == 3 and not date["certification"]:
                        for alt_date in release['release_dates']:
                            if alt_date["type"] == 4:
                                # We can create the dictionary with the correct keys
                                dictionary = {"id": movie_id, "iso_3166_1": release["iso_3166_1"], "certification": alt_date["certification"]}
                                cert_list.append(dictionary)
                                break
                    # Otherwise, include the certification for Type 3
                    elif date["type"] == 3:
                        # We can create the dictionary with the correct keys
                        dictionary = {"id": movie_id, "iso_3166_1": release["iso_3166_1"], "certification": date["certification"]}
                        cert_list.append(dictionary)

# Put the certification info into a DataFrame and CSV file for loading into the SQL database
cert2_df = pd.DataFrame(cert_list)
cert2_df.to_csv("certifications2.csv", index=False)
print(cert2_df.head())


    id iso_3166_1 certification
0   98         US             R
1   98         US             R
2   77         US             R
3  843         US            PG
4  641         US             R
